In [1]:
import sys
import os
home_dir = "/Users/mariappan.subramanian/Documents/"
sys.path.append(f'{home_dir}/repo/forked/meridian/meridian/')

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az
import IPython
import altair as alt
from IPython.display import HTML
import gc

from meridian.mpa.mpa_utils_meridian import MeridianMPAInput

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 25.8 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [3]:
from client_config import client_config, ec50_multiplier_config, media_parameters_config

In [4]:
coord_to_columns = load.CoordToColumns(
    time='time',
    geo='geo',
    controls=['sentiment_score_control', 'competitor_activity_score_control'],
    population='population',
    kpi='conversions',
    revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
    ],
    reach =['Channel3_reach'],
    frequency=['Channel3_frequency'],
    rf_spend=['Channel3_spend'],
)

In [5]:
correct_media_to_channel = {
    'Channel0_impression': 'Channel_0',
    'Channel1_impression': 'Channel_1',
    'Channel2_impression': 'Channel_2',
}
correct_media_spend_to_channel = {
    'Channel0_spend': 'Channel_0',
    'Channel1_spend': 'Channel_1',
    'Channel2_spend': 'Channel_2',
}

correct_reach_to_channel = {
    'Channel3_reach': 'Channel3',
}
correct_frequency_to_channel = {
    'Channel3_frequency': 'Channel3',
}
correct_rf_spend_to_channel = {
    'Channel3_spend': 'Channel3',
}

In [6]:
loader = load.CsvDataLoader(
    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_media_rf.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
    reach_to_channel=correct_reach_to_channel,
    frequency_to_channel=correct_frequency_to_channel,
    rf_spend_to_channel=correct_rf_spend_to_channel,
)
data = loader.load()

In [ ]:
roi_rf_mu = 0.2     # Mu for ROI prior for each RF channel.
roi_rf_sigma = 0.9  # Sigma for ROI prior for each RF channel.
prior = prior_distribution.PriorDistribution(
    roi_rf=tfp.distributions.LogNormal(roi_rf_mu, roi_rf_sigma, name=constants.ROI_RF)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

I0000 00:00:1754767860.199575 4776487 service.cc:148] XLA service 0x131211a10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754767860.199647 4776487 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1754767860.294065 4776487 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
run_model = True
file_path = f'{home_dir}/repo/forked/meridian/meridian/mpa/demo_rf_model_obj.pkl'
if run_model:
  mmm.sample_prior(500)
  mmm.sample_posterior(n_chains=2, n_adapt=1000, n_burnin=500, n_keep=1000, seed=1)
  model.save_mmm(mmm, file_path)
else:
  mmm=model.load_mmm(file_path)


2025-08-09 14:34:45.791336: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1754768086.910438 4776487 assert_op.cc:38] Ignoring Assert operator mcmc_retry_init/assert_equal_1/Assert/AssertGuard/Assert


In [11]:
mmm = model.load_mmm(file_path)

In [14]:
mmm